In [1]:
import datetime as dt
from datetime import datetime
import sys
import os
import boto3
import pandas as pd
from boto3.session import Session

from io import StringIO
import numpy as np
import shutil

# declaring some constants
bucket = 'arable-adse-dev'
session = Session()
client = session.client('s3')

import json
import psycopg2 as pg
import pandas.io.sql as psql
import warnings
warnings.filterwarnings('ignore')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
# get access to production data
client = boto3.client('secretsmanager')
response = client.get_secret_value(SecretId='alp-datasci')
a = json.loads(response['SecretString'])

def get_data_prod(query):
    connection = pg.connect(user = a['alp_user'],
                            password = a['alp_pword'],
                            host = a['alp_host'],
                            port = a['alp_port'],
                            database = a['alp_db'])

    df = psql.read_sql('''%s''' % (query), connection)

    connection.close()
    
    return df


client = boto3.client('secretsmanager')
response = client.get_secret_value(SecretId='han-db')
b = json.loads(response['SecretString'])

def get_data_test(query):    
    connection = pg.connect(user = b['user'],
                            password = b['alt_pword'],
                            host = b['alt_host'],
                            port = b['alt_port'],
                            database = b['alt_db'])

    df = psql.read_sql('''%s''' % (query), connection)

    connection.close()
    
    return df

In [10]:
# client = boto3.client('secretsmanager')
# response = client.get_secret_value(SecretId='alp-datasci')
# ds = json.loads(response['SecretString'])

# def get_data_prod(query):
#     connection = pg.connect(user = ds['alp_user'],
#                             password = ds['alp_pword'],
#                             host = ds['alp_host'],
#                             port = ds['alp_port'],
#                             database = ds['alp_db'])

#     df = psql.read_sql('''%s''' % (query), connection)

#     connection.close()
    
#     return df

In [14]:
device = 'C009846'
start = '2022-03-31'
end = '2022-06-23'

#### grab 5min and hourly data mark device

In [15]:
mark_5min = get_data_prod(f"""
        select c.*, r.lw_temp_3
        from device_data.calibrated c
        join device_data.raw r
        using(time, device)
        where c.device = '{device}'
        and time >= '{start}'
        and time <= '{end}'
        """)

## hourly data
hourly = get_data_prod(f"""
        select time, device, precip, rh
        from device_data.hourly
        where device = '{device}'
        and time >= '{start}'
        and time <= '{end}'
        """)

In [17]:
mark_5min.head()

,time,device,location,b1dw,b1uw,b2dw,b2uw,b3dw,b3uw,b4dw,...,tdew,dqs,lat,long,create_time,tair_version,tbelow_version,radcal_version,update_time,lw_temp_3
0,2022-03-31 00:03:26+00:00,C009846,620e876b655f54000d02dc70,0.002485,0.002793,0.001858,0.002709,0.002570,0.002565,0.001709,...,23.040890,None,9.69392,-82.98461,2022-03-31 00:22:40+00:00,20200128_1,None,20211208160343,2022-03-31 00:22:44+00,25.60
1,2022-03-31 00:08:27+00:00,C009846,620e876b655f54000d02dc70,0.002424,0.002533,0.001812,0.002926,0.002570,0.002627,0.001666,...,23.491903,None,9.69392,-82.98461,2022-03-31 00:22:40+00:00,20200128_1,None,20211208160343,2022-03-31 00:22:44+00,25.45
2,2022-03-31 00:13:28+00:00,C009846,620e876b655f54000d02dc70,0.002303,0.002403,0.001997,0.002655,0.002313,0.002565,0.001581,...,23.589300,None,9.69392,-82.98461,2022-03-31 00:22:40+00:00,20200128_1,None,20211208160343,2022-03-31 00:22:44+00,25.28
3,2022-03-31 00:18:29+00:00,C009846,620e876b655f54000d02dc70,0.002424,0.002793,0.001951,0.002601,0.002570,0.002565,0.001666,...,23.433271,None,9.69392,-82.98461,2022-03-31 00:22:44+00:00,20200128_1,None,20211208160343,2022-03-31 00:22:47+00,25.08
4,2022-03-31 00:23:30+00:00,C009846,620e876b655f54000d02dc70,0.002303,0.002468,0.001858,0.002818,0.002377,0.002941,0.001709,...,23.460278,None,9.69392,-82.98461,2022-03-31 01:22:39+00:00,20200128_1,None,20211208160343,2022-03-31 01:22:42+00,24.92


In [28]:
hourly = hourly.rename(columns={'rh':'rh_hourly'})

### join 5min and hourly data

In [30]:
mark_5min['time'] = mark_5min['time'].astype('datetime64[ns]')
mark_5min['hour'] = mark_5min['time'].dt.floor('h')

#hourly['time'] = hourly['time'].astype('datetime64[ns]')
#hourly['hour'] = hourly['time'].dt.floor('h')
hourly = hourly[['hour', 'precip', 'rh_hourly']]
mark = mark_5min.merge(hourly, on='hour', how='left')

In [31]:
mark

,time,device,location,b1dw,b1uw,b2dw,b2uw,b3dw,b3uw,b4dw,...,long,create_time,tair_version,tbelow_version,radcal_version,update_time,lw_temp_3,hour,precip,rh_hourly
0,2022-03-31 00:03:26,C009846,620e876b655f54000d02dc70,0.002485,0.002793,0.001858,0.002709,0.002570,0.002565,0.001709,...,-82.98461,2022-03-31 00:22:40+00:00,20200128_1,None,20211208160343,2022-03-31 00:22:44+00,25.60,2022-03-31 00:00:00,0.0,0.923392
1,2022-03-31 00:08:27,C009846,620e876b655f54000d02dc70,0.002424,0.002533,0.001812,0.002926,0.002570,0.002627,0.001666,...,-82.98461,2022-03-31 00:22:40+00:00,20200128_1,None,20211208160343,2022-03-31 00:22:44+00,25.45,2022-03-31 00:00:00,0.0,0.923392
2,2022-03-31 00:13:28,C009846,620e876b655f54000d02dc70,0.002303,0.002403,0.001997,0.002655,0.002313,0.002565,0.001581,...,-82.98461,2022-03-31 00:22:40+00:00,20200128_1,None,20211208160343,2022-03-31 00:22:44+00,25.28,2022-03-31 00:00:00,0.0,0.923392
3,2022-03-31 00:18:29,C009846,620e876b655f54000d02dc70,0.002424,0.002793,0.001951,0.002601,0.002570,0.002565,0.001666,...,-82.98461,2022-03-31 00:22:44+00:00,20200128_1,None,20211208160343,2022-03-31 00:22:47+00,25.08,2022-03-31 00:00:00,0.0,0.923392
4,2022-03-31 00:23:30,C009846,620e876b655f54000d02dc70,0.002303,0.002468,0.001858,0.002818,0.002377,0.002941,0.001709,...,-82.98461,2022-03-31 01:22:39+00:00,20200128_1,None,20211208160343,2022-03-31 01:22:42+00,24.92,2022-03-31 00:00:00,0.0,0.923392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21722,2022-06-22 23:37:44,C009846,62b0e0d20f3d0c000e82f72d,0.029624,0.002598,0.029683,0.003032,0.026146,0.002941,0.021445,...,-83.52443,2022-06-23 04:47:49+00:00,20200128_1,backup: lw_temp_3,20211208160343,2022-06-23 04:50:13+00,25.25,2022-06-22 23:00:00,0.0,0.924250
21723,2022-06-22 23:42:45,C009846,62b0e0d20f3d0c000e82f72d,0.023032,0.002663,0.022241,0.002926,0.019466,0.002690,0.015806,...,-83.52443,2022-06-23 04:47:49+00:00,20200128_1,backup: lw_temp_3,20211208160343,2022-06-23 04:50:13+00,25.07,2022-06-22 23:00:00,0.0,0.924250
21724,2022-06-22 23:47:46,C009846,62b0e0d20f3d0c000e82f72d,0.017030,0.002663,0.016165,0.002926,0.013618,0.002690,0.011193,...,-83.52443,2022-06-23 04:47:49+00:00,20200128_1,backup: lw_temp_3,20211208160343,2022-06-23 04:50:13+00,24.97,2022-06-22 23:00:00,0.0,0.924250
21725,2022-06-22 23:52:46,C009846,62b0e0d20f3d0c000e82f72d,0.011637,0.002728,0.011011,0.002872,0.009315,0.002565,0.007219,...,-83.52443,2022-06-23 04:47:55+00:00,20200128_1,backup: lw_temp_3,20211208160343,2022-06-23 04:48:09+00,24.89,2022-06-22 23:00:00,0.0,0.924250


#### calculate lfw

In [32]:
# 1 and 2
def get_lfw_current(df, which_tbelow):
    """Takes in df that have calibrated data, lw_temp3 from raw and outer joined with hourly precip
    Can be used to calculate lfw in 1 and 2, which 1 using tbelow instead of lw_temp_3
    """
    tdew_offset = 0.99
    low_rh_threshold = 0.90
    high_rh_threshold = 0.93
    
    if which_tbelow == 'lw_temp_3':
        # make tbelow_V1
        df['tbelow_corr'] = make_tbelow_v1(df.lw_temp_3)
        # make alter dpd with default offsets
        df['meta_lfw'] = alter_dew_point_depression(df, 'tbelow_corr', tdew_offset, low_rh_threshold)
    elif which_tbelow == 'tbelow':
        # make alter dpd
        df['meta_lfw'] = alter_dew_point_depression(df, 'tbelow', tdew_offset, low_rh_threshold)
    else:
        raise ValueError(f'not expecting tbelow type to be {which_tbelow}') 
    
    # cooperate ^ with hourly data to make final lfw
    return (df.precip > 0) | (df.rh_hourly > high_rh_threshold) | (df.meta_lfw > 0)

def make_tbelow_v1(sr):
    """Takes lw_temp_3 and returns Tbelow_corr"""
    za = np.array([0.01935698, 0.8618187, -2.34398788])
    return za[0] * (sr ** 2) + za[1] * sr + za[2]

def alter_dew_point_depression(df, tbelow_type, tdew_offset, low_rh_threshold):
    return (df[tbelow_type] <= df.tdew + tdew_offset) \
                    & (df.rh > low_rh_threshold)

In [35]:
tdf =mark.copy()
tdf['lfw'] = get_lfw_current(tdf, 'lw_temp_3')
tdf.tail()

,time,device,location,b1dw,b1uw,b2dw,b2uw,b3dw,b3uw,b4dw,...,tair_version,tbelow_version,radcal_version,update_time,lw_temp_3,hour,precip,rh_hourly,tbelow_corr,meta_lfw
21722,2022-06-22 23:37:44,C009846,62b0e0d20f3d0c000e82f72d,0.029624,0.002598,0.029683,0.003032,0.026146,0.002941,0.021445,...,20200128_1,backup: lw_temp_3,20211208160343,2022-06-23 04:50:13+00,25.25,2022-06-22 23:00:00,0.0,0.92425,31.758219,False
21723,2022-06-22 23:42:45,C009846,62b0e0d20f3d0c000e82f72d,0.023032,0.002663,0.022241,0.002926,0.019466,0.002690,0.015806,...,20200128_1,backup: lw_temp_3,20211208160343,2022-06-23 04:50:13+00,25.07,2022-06-22 23:00:00,0.0,0.92425,31.427764,False
21724,2022-06-22 23:47:46,C009846,62b0e0d20f3d0c000e82f72d,0.017030,0.002663,0.016165,0.002926,0.013618,0.002690,0.011193,...,20200128_1,backup: lw_temp_3,20211208160343,2022-06-23 04:50:13+00,24.97,2022-06-22 23:00:00,0.0,0.92425,31.244720,False
21725,2022-06-22 23:52:46,C009846,62b0e0d20f3d0c000e82f72d,0.011637,0.002728,0.011011,0.002872,0.009315,0.002565,0.007219,...,20200128_1,backup: lw_temp_3,20211208160343,2022-06-23 04:48:09+00,24.89,2022-06-22 23:00:00,0.0,0.92425,31.098563,False
21726,2022-06-22 23:57:47,C009846,62b0e0d20f3d0c000e82f72d,0.007274,0.002468,0.006550,0.002655,0.005268,0.002565,0.004015,...,20200128_1,backup: lw_temp_3,20211208160343,2022-06-23 04:48:09+00,24.76,2022-06-22 23:00:00,0.0,0.92425,30.861587,False


In [36]:
tdf =mark.copy()
tdf['lfw'] = get_lfw_current(tdf, 'tbelow')
tdf.tail()

,time,device,location,b1dw,b1uw,b2dw,b2uw,b3dw,b3uw,b4dw,...,create_time,tair_version,tbelow_version,radcal_version,update_time,lw_temp_3,hour,precip,rh_hourly,meta_lfw
21722,2022-06-22 23:37:44,C009846,62b0e0d20f3d0c000e82f72d,0.029624,0.002598,0.029683,0.003032,0.026146,0.002941,0.021445,...,2022-06-23 04:47:49+00:00,20200128_1,backup: lw_temp_3,20211208160343,2022-06-23 04:50:13+00,25.25,2022-06-22 23:00:00,0.0,0.92425,False
21723,2022-06-22 23:42:45,C009846,62b0e0d20f3d0c000e82f72d,0.023032,0.002663,0.022241,0.002926,0.019466,0.002690,0.015806,...,2022-06-23 04:47:49+00:00,20200128_1,backup: lw_temp_3,20211208160343,2022-06-23 04:50:13+00,25.07,2022-06-22 23:00:00,0.0,0.92425,True
21724,2022-06-22 23:47:46,C009846,62b0e0d20f3d0c000e82f72d,0.017030,0.002663,0.016165,0.002926,0.013618,0.002690,0.011193,...,2022-06-23 04:47:49+00:00,20200128_1,backup: lw_temp_3,20211208160343,2022-06-23 04:50:13+00,24.97,2022-06-22 23:00:00,0.0,0.92425,True
21725,2022-06-22 23:52:46,C009846,62b0e0d20f3d0c000e82f72d,0.011637,0.002728,0.011011,0.002872,0.009315,0.002565,0.007219,...,2022-06-23 04:47:55+00:00,20200128_1,backup: lw_temp_3,20211208160343,2022-06-23 04:48:09+00,24.89,2022-06-22 23:00:00,0.0,0.92425,True
21726,2022-06-22 23:57:47,C009846,62b0e0d20f3d0c000e82f72d,0.007274,0.002468,0.006550,0.002655,0.005268,0.002565,0.004015,...,2022-06-23 04:47:55+00:00,20200128_1,backup: lw_temp_3,20211208160343,2022-06-23 04:48:09+00,24.76,2022-06-22 23:00:00,0.0,0.92425,True


In [3]:
#device = 'C009846'

In [8]:
# df = get_data_prod(f"""
#         select * from device_data.calibrated
#         where device = '{device}'
#         """)

In [13]:
df.tail()

,time,device,location,b1dw,b1uw,b2dw,b2uw,b3dw,b3uw,b4dw,...,tbelow,tdew,dqs,lat,long,create_time,tair_version,tbelow_version,radcal_version,update_time
34832,2022-06-23 16:21:19+00:00,C009846,62b0e0d20f3d0c000e82f72d,1.364267,0.024033,1.815462,0.058323,1.387576,0.058419,1.467102,...,29.55,22.586409,None,10.28583,-83.52443,2022-06-23 16:45:46+00:00,20200128_1,None,20211208160343,2022-06-23 16:45:53+00
34833,2022-06-23 16:26:19+00:00,C009846,62b0e0d20f3d0c000e82f72d,0.893973,0.016108,1.152872,0.037543,0.886458,0.037153,0.918251,...,25.37,22.642298,None,10.28583,-83.52443,2022-06-23 16:45:46+00:00,20200128_1,None,20211208160343,2022-06-23 16:45:53+00
34834,2022-06-23 15:56:18+00:00,C009846,62b0e0d20f3d0c000e82f72d,0.439401,0.008573,0.533835,0.018091,0.434458,0.018454,0.419755,...,26.76,23.844526,None,10.28583,-83.52443,2022-06-23 16:45:42+00:00,20200128_1,backup: lw_temp_3,20211208160343,2022-06-23 16:46:15+00
34835,2022-06-23 16:01:18+00:00,C009846,62b0e0d20f3d0c000e82f72d,0.358582,0.007730,0.421782,0.014679,0.355226,0.014451,0.329544,...,27.45,23.702810,None,10.28583,-83.52443,2022-06-23 16:45:42+00:00,20200128_1,backup: lw_temp_3,20211208160343,2022-06-23 16:46:15+00
34836,2022-06-23 16:06:19+00:00,C009846,62b0e0d20f3d0c000e82f72d,0.761297,0.013834,0.977182,0.031898,0.776235,0.032398,0.784878,...,28.78,24.030708,None,10.28583,-83.52443,2022-06-23 16:45:42+00:00,20200128_1,backup: lw_temp_3,20211208160343,2022-06-23 16:46:15+00


In [6]:
# 1 and 2
def get_lfw_current(df, which_tbelow):
    """Takes in df that have calibrated data, lw_temp3 from raw and outer joined with hourly precip
    Can be used to calculate lfw in 1 and 2, which 1 using tbelow instead of lw_temp_3
    """
    tdew_offset = 0.99
    low_rh_threshold = 0.90
    high_rh_threshold = 0.93
    
    if which_tbelow == 'lw_temp_3':
        # make tbelow_V1
        df['tbelow_corr'] = make_tbelow_v1(df.lw_temp_3)
        # make alter dpd with default offsets
        df['meta_lfw'] = alter_dew_point_depression(df, 'tbelow_corr', tdew_offset, low_rh_threshold)
    elif which_tbelow == 'tbelow':
        # make alter dpd
        df['meta_lfw'] = alter_dew_point_depression(df, 'tbelow', tdew_offset, low_rh_threshold)
    else:
        raise ValueError(f'not expecting tbelow type to be {which_tbelow}') 
    
    # cooperate ^ with hourly data to make final lfw
    return (df.precip > 0) | (df.rh_hourly > high_rh_threshold) | (df.meta_lfw > 0)

def make_tbelow_v1(sr):
    """Takes lw_temp_3 and returns Tbelow_corr"""
    za = np.array([0.01935698, 0.8618187, -2.34398788])
    return za[0] * (sr ** 2) + za[1] * sr + za[2]

def alter_dew_point_depression(df, tbelow_type, tdew_offset, low_rh_threshold):
    return (df[tbelow_type] <= df.tdew + tdew_offset) \
                    & (df.rh > low_rh_threshold)

In [16]:
# tdf =df.copy()
# tdf['lfw'] = get_lfw_current(tdf, 'lw_temp_3')
# tdf.head()